# Extract Planck instrument parameters

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from the Planck NPIPE instrument models",
]

In [ ]:
import numpy as np
from astropy.io import fits

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!wget https://irsa.ipac.caltech.edu/data/Planck/release_3/ancillary-data/LFI_RIMO_R3.31.fits

In [ ]:
!wget https://portal.nersc.gov/cfs/cmb/planck2020/misc/PLANCK_RIMO_TF_R4.00.tar.gz

In [ ]:
!tar xzvf PLANCK_RIMO_TF_R4.00.tar.gz

In [ ]:
RIMO = dict(HFI = fits.open("simulated_maps/npipe_aux/RIMO_HFI_NPIPE.fits"), LFI = fits.open("simulated_maps/npipe_aux/RIMO_LFI_NPIPE.fits"))

In [ ]:
from pathlib import Path

In [ ]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [ ]:
table = QTable(
    names=["telescope", "band", "center_frequency", "fwhm", "nside", "bandpass_file", "beam_file"],
    dtype=[str, str, float, float, int, str, str],
    units=[None, None, u.GHz, u.arcmin, None, None, None],
)


In [ ]:
# Add rows for HFI and LFI
for instrument in ['LFI','HFI']:
    for band in RIMO[instrument]["FREQUENCY MAP PARAMETERS" if instrument == "HFI" else "FREQUENCY_MAP_PARAMETERS"].data:
        table.add_row(
            OrderedDict(
                telescope=instrument,
                band=band["FREQUENCY"],
                center_frequency=float(band["FREQUENCY"]) * u.GHz,
                fwhm=band["FWHM"] * u.arcmin,
                nside=2048 if instrument == "HFI" else 1024,
                bandpass_file="bandpass_" + band["FREQUENCY"] + ".tbl",
                beam_file="window_function_" + band["FREQUENCY"] + ".tbl",
            )
        )


In [ ]:
table

## Create bandpass files

In [ ]:
!wget http://sdc.uio.no/vol/cosmoglobe-data/BeyondPlanck/releases/v2/BP_bandpass_profiles/bp_corrected_30.dat
!wget http://sdc.uio.no/vol/cosmoglobe-data/BeyondPlanck/releases/v2/BP_bandpass_profiles/bp_corrected_44.dat
!wget http://sdc.uio.no/vol/cosmoglobe-data/BeyondPlanck/releases/v2/BP_bandpass_profiles/bp_corrected_70.dat

In [ ]:
RIMO["HFI"]["CHANNEL PARAMETERS"].data

In [ ]:
for band in ["030", "044", "070"]:
    corrected = np.loadtxt(f"bp_corrected_{band[1:]}.dat", unpack=True)
    plt.figure()
    plt.title(f"Band {band}")
    plt.semilogy(RIMO["LFI"]["BANDPASS_" + band].data["WAVENUMBER"], RIMO["LFI"]["BANDPASS_" + band].data["TRANSMISSION"], label="NPIPE")
    mask = corrected[1] > 1e-6
    plt.semilogy(corrected[0][mask], corrected[1][mask], label="BeyondPlanck")


In [ ]:

from astropy import constants as const

In [ ]:
ch

In [ ]:
skip = {"217":2, "353":3, "545":9, "857":12}
bandpass = {}
for ch in table:
    band = ch["band"]
    if ch["telescope"] == "HFI":
        freq = (RIMO["HFI"]["BANDPASS_F"+band].data["WAVENUMBER"] * (1/u.cm) * const.c).to(u.GHz)
        weight = RIMO["HFI"]["BANDPASS_F"+band].data["TRANSMISSION"]
        mask = np.logical_and(freq > .5*ch["center_frequency"], freq < 1.5 * ch["center_frequency"])
        mask = np.logical_and(mask, weight > 1e-6)
        freq = freq[mask]
        weight = weight[mask]
        if band in skip:
            freq = freq[::skip[band]]
            weight = weight[::skip[band]]
    else:


        corrected = np.loadtxt(f"bp_corrected_{band[1:]}.dat", unpack=True)
        mask = corrected[1] > 1e-6
        freq = corrected[0][mask]
        weight = corrected[1][mask]
    bandpass[band] = QTable( names ="bandpass_frequency bandpass_weight".split(),
                            units=[u.GHz, None],
                            data=[freq, weight])
    bandpass[band].meta["comments"] = ipac_warning
    bandpass[band].write(f"bandpass_{band}.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
for row in table:
    if row["telescope"] == "HFI":
        plt.semilogy(
            bandpass[row["band"]]["bandpass_frequency"],
            bandpass[row["band"]]["bandpass_weight"],
            label=f"HFI {row['band']} ({len(bandpass[row['band']])})"
        )
plt.xlabel("Frequency (GHz)")
plt.ylabel("Transmission")
plt.title("HFI Channels Bandpasses (number of points)")
plt.legend()
plt.show()
